<a href="https://colab.research.google.com/github/HayatoYagi/Elliptic-Curves/blob/main/elliptic_curve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import math

def divisors(n):
    res = []
    i = 1
    while(i * i < n):
        if n % i == 0:
            res.append(i)
            res.append(n // i)
        i += 1
    if i * i == n:
        res.append(i)
    return res

class EllipticCurve:
    """
    y^2 = f(x) := x^3 + ax^2 + bx + c
    """
    def __init__(self, a, b, c):
        self._a = a
        self._b = b
        self._c = c
        # todo: 非特異であることを確認する（実装ムズそう）
    
    def f(self, x):
        return (x * x * x
                + self._a * x * x
                + self._b * x
                + self._c)
        
    def f_prime(self, x):
        return (3 * x * x
                + 2 * self._a * x
                + self._b)

    def discriminant(self):
        return (-4 * self._a * self._a * self._a * self._c
                + self._a * self._a * self._b * self._b
                + 18 * self._a * self._b * self._c
                - 4 * self._b * self._b * self._b
                - 27 * self._c * self._c)

    def torsion_candidates(self):
        """
        有限位数の有理点の候補を返す。
        必要条件しか確認していないことに注意。
        """
        def y_candidates():
            res = [0]
            D = self.discriminant()
            assert(D < 0)
            i = 1
            while i * i <= -D:
                if(-D % (i * i) == 0):
                    res.append(i)
                i += 1
            return res

        def find_x(y):
            res = []
            a = self._a
            b = self._b
            c = self._c - y * y
            if c == 0:
                res.append(0)
                if b == 0:
                    if(a != 0):
                        res.append(-a)
                else:
                    for d in divisors(abs(b)):
                        if(d * d + a * d + b == 0):
                            res.append(d)
                        if(d * d - a * d + b == 0):
                            res.append(-d)
            else:
                for d in divisors(abs(c)):
                    if(d * d * d + a * d * d + b * d + c == 0):
                        res.append(d)
                    if(- d * d * d + a * d * d - b * d + c == 0):
                        res.append(-d)
            return res

        res = []
        for y in y_candidates():
            for x in find_x(y):
                res.append((x,y))
                if(y != 0):
                    res.append((x,-y))
        return res
    
    def double_x(self, x):
        """
        入力された曲線上の整数点のx座標に対し、その点を2倍した点のx座標とそれが整数かどうかを返す。
        """
        numer = (x * x * x * x
                 - 2 * self._b * x * x
                 - 8 * self._c * x
                 + self._b * self._b
                 - 4 * self._a * self._c)
        denom = 4 * self.f(x)
        return (numer / denom, numer % denom == 0)

    def add(self, p1, p2):
        if math.isinf(p1):
            # todo
            return -1
        x1,y1 = p1
        x2,y2 = p2
        if x1 == x2:
            if y1 == y2:
                la = self.f_prime(x1) / 2 / y1
                nu = y1 - la * x1
                x3, _ = self.double_x(x1)
                y3 = la * x3 + nu
                return (x3, -y3)
            else:
                return math.inf
        else:
            la = (y2 - y1) / (x2 - x1) # lambda
            nu = y1 - la * x1 # nu
            x3 = la * la - self._a - x1 - x2
            y3 = la * x3 + nu
            return (x3, -y3)

    def scalar_multiple(self, p, n:int):
        assert(n >= 1)
        if(n == 1):
            return p
        q = self.scalar_multiple(p, n // 2)
        q = self.add(q, q)
        if(n % 2 == 1):
            q = self.add(q, p)
        return q

In [30]:
e1 = EllipticCurve(0,-43,166)
print(e1.torsion_candidates())
e1.p1 = (3,8)
e1.add(e1.p1, e1.p1)
for i in range(1, 8):
    print(i, e1.scalar_multiple(e1.p1, i))

[(3, 8), (3, -8), (-5, 16), (-5, -16), (11, 32), (11, -32)]
1 (3, 8)
2 (-5.0, -16.0)
3 (11.0, -32.0)
4 (11.0, 32.0)
5 (-5.0, 16.0)
6 (3.0, -8.0)
7 inf


In [16]:
e2 = EllipticCurve(0,0,1)
print(e2.torsion_candidates())
print(e2.double_x(2))

0 [-1]
1 [0]
3 [2]
[(-1, 0), (0, 1), (0, -1), (2, 3), (2, -3)]
(0.0, True)


In [27]:
e3 = EllipticCurve(0,0,4)
print(e3.torsion_candidates())
e3.p1 = (0,2)
e3.p2 = (0,-2)
print(e3.add(e3.p1, e3.p1))
print(e3.add(e3.p1, e3.p2))
for i in range(1,6):
    print(i, e3.scalar_multiple(e3.p1, i))

[(0, 2), (0, -2)]
(0.0, -2.0)
inf
1 (0, 2)
2 (0.0, -2.0)
3 inf
4 (0.0, 2.0)
5 (0.0, -2.0)
